In [145]:
from helper import *
from filenames_and_paths import *

# Назначение нормативных карт

In [147]:
filename = filenames014[0]
path = path014
th_filename = filename+"_th"
folders.end_folder = path
raw = mne.io.read_raw_eeglab(folders.preprocessed_data + folders.end_folder + filenames014[0] + '.set')
mhw = MicrostateHelperWrapper(folders, raw, th_filename)
mhw = mhw.load()

In [148]:
global_maps = pd.read_csv(folders.save_data + "global_maps.csv")
global_maps

In [149]:
mhw.calc_normative_labels(global_maps)

In [150]:
ms_clone = mhw.ms.copy()
# ms_clone["Sequence"]= list(map((lambda x: remapper[x]), mhw.ms["Sequence"]))
# ms_clone

In [151]:
mhw.normative_labels

In [152]:
nk.microstates_plot(mhw.ms, epoch=(0, 2048*3))

In [153]:
print(mhw.normative_labels.T[0].label)
print(mhw.normative_labels.T[1].label)
print(mhw.normative_labels.T[2].label)
print(mhw.normative_labels.T[3].label)


In [154]:
def create_remapper(normative_labels):
    label_idx_remapper = {
        "A": 0,
        "B": 1,
        "C": 2,
        "D": 3
    }
    idx_idx_remapper = {}
    for i in range(4):
        idx_idx_remapper[i] = label_idx_remapper[normative_labels[i]["label"]]
    return idx_idx_remapper
remapper = create_remapper(mhw.normative_labels.T)
remapper

In [189]:
def reorder_microstates(ms, remapper, normative_labels):
    ms_clone_1 = copy.deepcopy(ms)
    for i in range(len(ms["Sequence"])):
        ms_clone_1["Sequence"][i] = remapper[ms["Sequence"][i]]
    ms_array_clone = ms_clone_1["Microstates"].copy()
    for i in range(4):
        ms_clone_1["Microstates"][i] = (ms_array_clone[remapper[i]] * (-1 if normative_labels.T[remapper[i]].inv else 1))

    return ms_clone_1

In [190]:
ms_clone_reordered = reorder_microstates(ms_clone, remapper, mhw.normative_labels)
# print(ms_clone_reordered["Microstates"])

In [191]:
# print(mhw.ms["Microstates"])

In [192]:
# print(mhw.ms["Sequence"][500:1000])

In [193]:
# print(ms_clone_reordered["Sequence"][500:1000])

In [194]:
nk.microstates_plot(mhw.ms, epoch=(0, 2048*3))

In [195]:
nk.microstates_plot(ms_clone_reordered, epoch=(0, 2048*3))

In [196]:
nk.microstates_static(ms_clone, show=True)

In [197]:
nk.microstates_static(ms_clone_reordered, show=True)